In [1]:
from pyspark.sql import SparkSession

# New API
spark = SparkSession\
        .builder\
        .master("local[1]")\
        .appName("team22")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .getOrCreate()
        
# Old API (RDD)
sc = spark.sparkContext

In [2]:
flight_data = spark\
    .read\
    .format("csv")\
    .option("header","true")\
    .load("hdfs://192.168.1.153:9000/team22/data/1987.csv")

In [3]:
flight_data.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CarrierDelay:

In [4]:
print(flight_data.count())

1311826


In [5]:
flight_data = flight_data.drop("CancellationCode","Distance","AirTime","CRSElapsedTime","Diverted","SecurityDelay","LateAircraftDelay","UniqueCarrier","TaxiIn", "TaxiOut", "FlightNum", "TailNum","ActualElapsedTime","CarrierDelay" ,"WeatherDelay","NASDelay")

In [6]:
flight_data.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Cancelled: string (nullable = true)



# Grandma Question

Filter Origin and Destination

In [20]:
airports_Q1 = flight_data.filter(flight_data["Origin"]=="LAX").filter(flight_data["Dest"]=="HNL")

['BGM',
 'MSY',
 'GEG',
 'BUR',
 'SNA',
 'GTF',
 'GRB',
 'IDA',
 'GRR',
 'EUG',
 'PSG',
 'GSO',
 'PVD',
 'MYR',
 'ISO',
 'OAK',
 'MSN',
 'FAR',
 'BTM',
 'SCC',
 'DCA',
 'CID',
 'MLU',
 'WRG',
 'HLN',
 'RDM',
 'LEX',
 'ORF',
 'EVV',
 'KTN',
 'CRW',
 'CWA',
 'SAV',
 'TRI',
 'CDV',
 'CMH',
 'SPN',
 'CAK',
 'UCA',
 'CHO',
 'MOB',
 'PNS',
 'LIH',
 'IAH',
 'HNL',
 'ERI',
 'SHV',
 'SJC',
 'CVG',
 'TOL',
 'LGA',
 'BUF',
 'TLH',
 'HPN',
 'RDD',
 'AUS',
 'MLI',
 'SJU',
 'ATW',
 'LGB',
 'GJT',
 'AVL',
 'LYH',
 'BFL',
 'GFK',
 'RNO',
 'SRQ',
 'SBN',
 'EYW',
 'RST',
 'JAC',
 'CHS',
 'TUL',
 'RSW',
 'HRL',
 'ISP',
 'AMA',
 'BOS',
 'MAF',
 'MLB',
 'EWR',
 'LAS',
 'BIS',
 'FAI',
 'JAN',
 'DLH',
 'DEN',
 'ALB',
 'CPR',
 'LNK',
 'OME',
 'IAD',
 'PSP',
 'SBA',
 'BOI',
 'DRO',
 'SEA',
 'CMI',
 'LAN',
 'MCI',
 'PIR',
 'FLG',
 'BNA',
 'CLT',
 'PSC',
 'BLI',
 'ORH',
 'ABQ',
 'PBI',
 'PIE',
 'SDF',
 'SCK',
 'ACV',
 'MRY',
 'BDL',
 'DAL',
 'ITH',
 'CLE',
 'APF',
 'TVL',
 'PDX',
 'MFR',
 'MIA',
 'ILG',
 'TPA',


Week day delay

In [27]:
week_day_delay = airports_Q1.select("DayOfWeek", "ArrDelay", "DepDelay")
week_day_delay = week_day_delay.withColumn("TotalDelay", week_day_delay["ArrDelay"].cast("float")+week_day_delay["DepDelay"].cast("float"))
# week_day_delay = week_day_delay.groupBy("DayOfWeek")
week_day_delay = week_day_delay.select("DayOfWeek","TotalDelay").sort("DayOfWeek").groupBy("DayOfWeek").count()
# week_day_delay.take(5)

Monthly delay

In [23]:
month_delay = airports_Q1.select("DayofMonth", "ArrDelay", "DepDelay")
month_delay = month_delay.withColumn("Total delay", month_delay["ArrDelay"].cast("float")+month_delay["DepDelay"].cast("float"))
month_delay = month_delay.groupBy("DayofMonth")

# Least delays/flights

In [ ]:
airports_Q2 = flight_data.select("Origin", "DepDelay")